### Importing required libraries

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from typing import List, Dict

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset

 [Dataset](https://www.kaggle.com/mousehead/songlyrics/data#).

This dataset contains name, artist, and lyrics for *57650 songs in English*. The data has been acquired from LyricsFreak through scraping.

In [12]:
# /content/drive/MyDrive/Colab Notebooks/music_recommender-master/content_based_recommedation_system/songdata.csv
songs = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/music_recommender-master/content_based_recommedation_system/songdata.csv')

In [ ]:
songs = pd.read_csv('content based recommedation system/songdata.csv')

In [13]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Because of the dataset being so big, we are going to resample only 5000 random songs.

In [14]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

We can notice also the presence of `\n` in the text, so we are going to remove it.

In [15]:
songs['text'] = songs['text'].str.replace(r'\n', '')

<ipython-input-15-215fe46be90d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each song lyric, word-by-word.

Here, we pay particular attention to the arguments we can specify.

In [16]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [17]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

*How do we use this matrix for a recommendation?*

We now need to calculate the similarity of one lyric to another. We are going to use **cosine similarity**.

We want to calculate the cosine similarity of each item with every other item in the dataset. So we just pass the lyrics_matrix as argument.

In [18]:
cosine_similarities = cosine_similarity(lyrics_matrix)

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [19]:
similarities = {}

In [20]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

After that, all the magic happens. We can use that similarity scores to access the most similar items and give a recommendation.

For that, we'll define our Content based recommender class.

In [21]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)

        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

Now, instantiate class

In [22]:
recommedations = ContentBasedRecommender(similarities)

Then, we are ready to pick a song from the dataset and make a recommendation.

In [23]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4
}

In [24]:
recommedations.recommend(recommendation)

The 4 recommended songs for Heaven Knows are:
Number 1:
Down On Love by Foreigner with 0.176 similarity score
--------------------
Number 2:
Say That You Love Me by Kyla with 0.173 similarity score
--------------------
Number 3:
Are You Sincere by Andy Williams with 0.169 similarity score
--------------------
Number 4:
You'll Lose A Precious Love by The Temptations with 0.164 similarity score
--------------------


And we can pick another random song and recommend again:

In [27]:
recommendation2 = {
    "song": songs['song'].iloc[130],
    "number_songs": 4
}

In [26]:
recommedations.recommend(recommendation2)

The 4 recommended songs for In The Name Of Love are:
Number 1:
Try Me, I Know We Can Make It by Donna Summer with 0.246 similarity score
--------------------
Number 2:
Try A Little Tenderness by Otis Redding with 0.232 similarity score
--------------------
Number 3:
Searching by INXS with 0.208 similarity score
--------------------
Number 4:
I Will Always Love You by Linda Ronstadt with 0.207 similarity score
--------------------
